In [ ]:
# import libraries
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [ ]:
# create selenium session
options = Options()
options.add_argument("--headless")
options.add_argument('<user profile>')
driver = webdriver.Chrome(executable_path='<path to chrome driver>', chrome_options=options)
driver.maximize_window()
driver.implicitly_wait(10)
driver.set_script_timeout(10000000)

# or do this everytime
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# accept cookies
driver.get(url)
driver.find_element("xpath",'//*[@id="cmpwelcomebtnyes"]/a').click()    

In [ ]:
# infinite scrolling
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
# click button in pop-up to go to the site
url = 'https://www.autoscout24.de/detailsuche?sort=standard&desc=0&ustate=N,U&atype=C&cy=D&ocs_listing=include'
driver.get(url)
wait = WebDriverWait(driver, 20)
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "gdpr-consent-notice")))
button = driver.find_element('id','save')
driver.execute_script("arguments[0].click();", button)
driver.switch_to.parent_frame()
driver.find_element('xpath','//*[@id="__next"]/div/div/div[2]/div[2]/div[1]/div/div[1]/div[2]/div[1]/div/div[1]/div/div[1]/div/button').click()
soup = BeautifulSoup(driver.page_source, 'html.parser')

### Beautiful Soup examples

In [ ]:
url = '....'
driver.get(url)

# Beautifyul Soup example
soup = BeautifulSoup(driver.page_source, 'html.parser')
product_name = soup.find('h1', class_='sale-box__product-name').get_text().strip()
price_box = soup.find('div', class_='sale-box__price-wrapper')

if price_box.find('span', class_='sale-box__price sale-box__price--base'):
    price = price_box.find('span', class_='sale-box__price sale-box__price--base').get_text().replace('€', '').strip()

# if you want to check for attribute
name = article.find('h2').get_text()
if article.has_attr('data-lazyloaddata'):
    try:
        lazy = json.loads(article.get('data-lazyloaddata'))
        if "branche" in lazy:
            branch = lazy["branche"]
        if "adresseKompakt" in lazy:
            address_fields = lazy["adresseKompakt"]
            if 'stadtteil' in address_fields:
                district = address_fields["stadtteil"]                            
            if "telefonnummer" in address_fields:
                telefonnummer = address_fields["telefonnummer"]
            if "plzOrt" in address_fields:
                plzOrt = address_fields["plzOrt"]
                zip_code = re.findall(r'\d+',plzOrt)
                if zip_code:
                    zip_code = zip_code[0]
                else:
                    zip_code = '' 
                city = plzOrt.replace(zip_code,'').strip()
            if "strasseHausnummer" in address_fields:
                street = address_fields["strasseHausnummer"]
            address = street + ' ' + plzOrt + ' ' + district

# attribute condition
addr = article.find('address').find('p',{'data-wipe-name':'Adresse'})

# load more button (click in loop) like
loadmore = driver.find_element('id',"mod-LoadMore--button")
try:
    while loadmore.is_displayed():
        loadmore.click()
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all('article', class_='mod mod-Treffer')
        print(len(articles))
        with open('articles.txt', 'wb') as fp:
            pickle.dump(str(soup), fp)

except Exception as e:
    print(e)


# Scraping a table
table1 = soup.find('table')

headers = []
for i in table1.find_all('th'):
    title = i.text.strip()
    headers.append(title)

mydata = pd.DataFrame(columns = headers)
for j in table1.find_all('tr')[1:]:
    row_data = j.find_all('td')
    if row_data:
        row = [i.text.strip() for i in row_data]
        length = len(mydata)
        mydata.loc[length] = row
mydata.to_excel(filename, index=False)

### Selenium Examples

In [ ]:
# Selenium example
filter_box = Select(driver.find_element('xpath', '//*[@id="selectProv-33c9327b-8dd7-11ea-b27f-005056a48f82-00055___"]'))
for opt in filter_box.options:
	province = opt.text
	filter_box.select_by_visible_text(province)
	time.sleep(1)
# wait for element to be clickable
	WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="centercontainer"]/div[2]/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div[1]/a'))).click()
	cards = driver.find_element('xpath', '//*[@id="mapAccordion_33c9327b-8dd7-11ea-b27f-005056a48f82-00055____listado"]').find_elements(By.CLASS_NAME, 'card')
	for card in cards:
		card_btn = card.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'button')
		name = card_btn.text
		card_btn.click()
		card_body = card.find_element(By.CSS_SELECTOR, 'div.card-body.bg-traffic').find_elements(By.CSS_SELECTOR, 'div.col-12.col-md-6.text-to-resize')
		contact = card_body[0].find_element(By.CSS_SELECTOR, 'div.col-12.col-md-9.text-to-resize').find_elements(By.CLASS_NAME, 'fs-14')
		for c in contact:
			time.sleep(2)
			try:
				x = c.find_element(By.TAG_NAME, 'a').text.strip()
				if x and 'null' not in x:
					if '@' in x:
						email = x
					else:
						phone = x
			except:
				pass
		address = card_body[1].find_element(By.CSS_SELECTOR, 'div.col-12.col-md-9.text-to-resize').find_elements(By.CLASS_NAME, 'fs-14')
		street = address[0].text


In [ ]:
# selenium example select options then submit and scrape resulting page

for combos in make_model_combinations:
    hersteller, modell, motor = combos
    driver.get(url)
    select_hersteller = Select(driver.find_element('xpath', '//*[@id="stepbystep0"]'))
    select_hersteller.select_by_visible_text(hersteller)
    time.sleep(0.5)
    select_modell = Select(driver.find_element('xpath', '//*[@id="stepbystep1"]'))
    select_modell.select_by_visible_text(modell)
    time.sleep(0.5)
    select_motor = Select(driver.find_element('xpath', '//*[@id="stepbystep2"]'))
    select_motor.select_by_visible_text(motor)
    time.sleep(0.5)
    driver.find_element('xpath','//*[@id="vehiclecontext_stepbystep"]/div[4]/button').click()
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = soup.find_all('li', class_='item-product')
    for product in products:
        title = product.find('div', class_='s1 title').get_text().strip()
        note_price = product.find('span', class_='note-price').get_text().strip()
        price_big = product.find('span', class_='price-big').get_text().strip()
        items.append([hersteller, modell, motor, title, note_price, price_big])

In [ ]:
# enter value in text box example

input = driver.find_element('xpath','//*[@id="make-input-primary-filter"]')
input.send_keys(make)
input.send_keys(Keys.ENTER)
        

### Cloud Scraper example

In [ ]:
import cloudscraper
scraper = cloudscraper.create_scraper()

url = '...'
html_home_page = scraper.get(url).text
soup = BeautifulSoup(html_home_page, 'html.parser')
articles = soup.find_all('div', class_='...')
for article in articles:
    detail_site = article.get('onclick').replace('location.href=','')[1:-1]
    html_article = scraper.get(detail_site).text
    article_soup = BeautifulSoup(html_article, 'html.parser')
    # and so on...

### Decode encoded string

In [ ]:
def cfDecodeEmail(encodedString):
    r = int(encodedString[:2],16)
    email = ''.join([chr(int(encodedString[i:i+2], 16) ^ r) for i in range(2, len(encodedString), 2)])
    return email

email = article_soup.find('div', id= 'cp-email').find('span', class_='__cf_email__').get('data-cfemail')
email = cfDecodeEmail(email)

In [ ]:
driver.quit()

### Random sleeps

In [ ]:
from random import randint
from time import sleep

# Sleep for random (1-5) seconds
sleep(randint(1,5))